In [1]:
!pip install https://huggingface.co/spacy/en_core_web_trf/resolve/main/en_core_web_trf-any-py3-none-any.whl


In [2]:
pip install xlsxwriter

In [3]:
pip install openpyxl

In [4]:

import sys
sys.path.append('../input')
import nltk
import spacy
import random
import itertools
from xuexibao.utils.fileUtils import *
random.seed('data')
nlp = spacy.load('en_core_web_trf')




from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# lemmatization
wnl = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [5]:
def get_n_gram(text_list,n):
    grams = []
    m = len(text_list) - n + 1
    for idx in range(m):
        gram = text_list[idx:idx+n]
        grams.append(gram)
    return grams
# sample_text = 'so much for making this app such an enjoyable'.split()
# get_n_gram(sample_text,1)
# get_n_gram(sample_text,2)
# get_n_gram(sample_text,3)

In [6]:
def pos_tag_map(pos_tag):
    if pos_tag.startswith('N'):
        return 'N'
    elif pos_tag.startswith('V'):
        return 'V'
    elif pos_tag.startswith('J'):
        return 'J'
    elif pos_tag in ['PRP','PRP$','WP','WP$']:
        return 'P'
    else:
        return pos_tag

In [7]:
def text_processing(text):
    sentences = nltk.sent_tokenize(text)
    sent_lists = []
    for idx,sent in enumerate(sentences):
        doc = nlp(sent)
        # sent_list = [(wnl.lemmatize(w,get_wordnet_pos(t)).lower(),pos_tag_map(t)) for w,t in nltk.pos_tag(nltk.word_tokenize(sent))]
        sent_list = [(token.lemma_, pos_tag_map(token.tag_), token.i, token.text)for token in doc]
        sent_lists.append(sent_list)
    return sent_lists

In [8]:

patterns = [['N','N'],
           ['V','N'],
           ['J','N'],
           ['N','CC','N'],
           ['J','N','N'],
           ['N','N','N'],
           ['V','P','N'],
           ['V','N','N'],
           ['V','J','N'],
           ['J','J','N'],
           ['N','IN','N'],
           ['V','DT','N'],
           ['V','N','IN','N'],
           ['J','N','N','N'],
           ['J','CC','J'],
           ['V','IN','J','N'],
            ['V','P','J','N'],
           ['N','CC','N','N']]

patterns_dict = {k+1:v for k,v in enumerate(patterns)}

In [9]:
def delete_features(features):
    delete_list = []
    for item in itertools.combinations(features, 2):
        if set(item[0][2]).issubset(item[1][2]):
            delete_list.append(item[0])
        elif set(item[1][2]).issubset(item[0][2]):
            delete_list.append(item[1])

    new_features = []
    
    for value in features:
        if value not in delete_list:
            new_features.append(value)
    return new_features

In [10]:
def fiter(lines):
    new_lines = []
    for i in lines:
        if len(i[0])>60 and len(i[0])<600 and "Incredible" not in i[0] and i not in new_lines:
            new_lines.append(i)
    return new_lines
def patern(test_lines,name):
    json_list = []
    for ridx,line in enumerate(test_lines):
        tokenized_sents = text_processing(line[0])
        for sidx,tokenized_sent in enumerate(tokenized_sents):
            result_dict = {'review_id':ridx,'sent_id':sidx,'sent':' '.join([item[-1] for item in tokenized_sent]),}
            features = []
            gram_2 = get_n_gram(tokenized_sent, 2)
            gram_3 = get_n_gram(tokenized_sent, 3)
            gram_4 = get_n_gram(tokenized_sent, 4)
            for item in gram_2 + gram_3 + gram_4:
                pos_list = []
                word_list = []
                idx_list = []
                for word_tuple in item:
                    word, pos, idx, _ = word_tuple
                    pos_list.append(pos)
                    word_list.append(word)
                    idx_list.append(idx)
                for k,pattern in patterns_dict.items():
                    if pos_list == pattern:
                        features.append([word_list, pos_list,idx_list])

            new_features = delete_features(features)
            result_dict['features'] = new_features
            json_list.append(result_dict)

    print(len(json_list))
    dumpJson('./{}.json'.format(name), json_list)

In [11]:
import random
random.seed('data')
app_list = [
"Facebook_privacy",
"Instagram_privacy",
"Spotify_privacy",
"TikTok_privacy",
"Twitter_privacy",
"YouTube_privacy",
"pocketguard"
]

for i in app_list:
    lines = readCsvToList('../input/privacy-datas/Privacy_data/{}.csv'.format(i))
    a = fiter(lines)[:500]
    random.shuffle(a)
    test_lines = a[:50]
    patern(test_lines,i+'_pos')